# Wave Equation

\begin{equation}
\begin{aligned}
    \partial_t^2 u(t,x) &= \partial_x^2 u(t,x), && t \in (0,T), x \in (x_{min}, x_{max}), \\
    u(0,x) &= u_0(x), && x \in (x_{min}, x_{max}), \\
    \partial_t u(0,x) &= v_0(x), && x \in (x_{min}, x_{max}), \\
    \text{boundary conditions}, &&& x \in \partial (x_{min}, x_{max}).
\end{aligned}
\end{equation}



In [ ]:
using Revise
using SummationByPartsOperators, OrdinaryDiffEq
using Plots, LaTeXStrings, Printf

# general parameters
xmin = -1.
xmax = +1.
tspan = (0., 8.0)
u0func(x) = exp(-20x^2)
du0func(x) = zero(x)
# HomogeneousNeumann, HomogeneousDirichlet, NonReflecting BCs are implemented up to now
left_bc  = Val(:HomogeneousNeumann)
right_bc = Val(:HomogeneousDirichlet)

# discretisation parameters
interior_order = 4
N = 101

# setup spatial semidiscretisation
D2 = derivative_operator(MattssonSvärdShoeybi2008(), 2, interior_order, xmin, xmax, N)
semidisc = WaveEquationNonperiodicSemidiscretisation(D2, left_bc, right_bc)
ode = semidiscretise(du0func, u0func, semidisc, tspan)

# solve ode
sol = solve(ode, DPRKN6(), dt=0.25*D2.Δx, adaptive=false, 
            saveat=range(first(tspan), stop=last(tspan), length=200))

# visualise the result
plot(xguide=L"x")
plot!(evaluate_coefficients(sol[end].x[2], semidisc), label=L"u")
plot!(evaluate_coefficients(sol[end].x[1], semidisc), label=L"\partial_t u")

In [ ]:
# make a movie
anim = Animation()
idx = 1
x, u = evaluate_coefficients(sol[idx].x[2], semidisc)

fig = plot(x, u, xguide=L"x", yguide=L"u", xlim=extrema(x), ylim=(-1., 1.),
           #size=(1024,768), dpi=250,
           label="", title=@sprintf("\$t = %6.2f \$", sol.t[idx]))
for idx in 1:length(sol.t)
    fig[1] = x, sol.u[idx].x[2]
    plot!(title=@sprintf("\$t = %6.2f \$", sol.t[idx]))
    frame(anim)
end
gif(anim)

## 2D

Since lazy tensor/Kronecker products are not implemented yet,
the operators are converted to sparse matrices via `LinearMaps.jl`.

In [ ]:
using LinearMaps
using SparseArrays, LinearAlgebra

# general parameters
xmin = -1.
xmax = +1.
ymin = -1.
ymax = +1.
tspan = (0., 4.0)
u0func(x,y) = exp(-20*(x^2+y^2))
du0func(x,y) = zero(x)
# HomogeneousNeumann, HomogeneousDirichlet BCs are implemented up to now
# NonReflecting requires the values of du and is not usable in the simplified approach below
xmin_bc = Val(:HomogeneousDirichlet)
xmax_bc = Val(:HomogeneousDirichlet)
ymin_bc = Val(:HomogeneousNeumann)
ymax_bc = Val(:HomogeneousNeumann)

# discretisation parameters
interior_order = 4
Nx = 151
Ny = 151

# setup spatial semidiscretisation
D2x = derivative_operator(MattssonSvärdShoeybi2008(), 2, interior_order, xmin, xmax, Nx)
semidiscx = WaveEquationNonperiodicSemidiscretisation(D2x, xmin_bc, xmax_bc)
opx = LinearMap{Float64}((ddu,u)->semidiscx(ddu,zero(u),u,nothing,0.), Nx, ismutating=true) |> sparse

D2y = derivative_operator(MattssonSvärdShoeybi2008(), 2, interior_order, ymin, ymax, Ny)
semidiscy = WaveEquationNonperiodicSemidiscretisation(D2y, ymin_bc, ymax_bc)
opy = LinearMap{Float64}((ddu,u)->semidiscy(ddu,zero(u),u,nothing,0.), Ny, ismutating=true) |> sparse

x = SummationByPartsOperators.grid(D2x)
y = SummationByPartsOperators.grid(D2y)
op = kron(opx, sparse(I, size(opy)...)) + kron(sparse(I, size(opx)...), opy)
u0 = u0func.(x, y') |> vec
du0 = du0func.(x, y') |> vec
ode = SecondOrderODEProblem((ddu,du,u,p,t)->mul!(ddu,op,u), du0, u0, tspan)

# solve ode
sol2D = solve(ode, DPRKN6(), dt=0.25*min(D2x.Δx,D2y.Δx), adaptive=false, 
              saveat=range(first(tspan), stop=last(tspan), length=200))

# visualise the result
plot(xguide=L"x", yguide=L"y", clim=(-1.,1.), aspect_ratio=:equal, size=(600,520))
heatmap!(x, y, sol2D[end].x[2], c=:bluesreds)

In [ ]:
# make a movie
anim = Animation()
idx = 1

fig = heatmap(x, y, sol2D.u[idx].x[2], xguide=L"x", yguide=L"y", clim=(-1., 1.), c=:bluesreds,
              aspect_ratio=:equal, size=(600,520),
               #size=(1024,768), dpi=250,
               label="", title=@sprintf("\$t = %6.2f \$", sol2D.t[idx]))
for idx in 1:length(sol.t)
    fig[1] = x, y, sol2D.u[idx].x[2]
    heatmap!(title=@sprintf("\$t = %6.2f \$", sol2D.t[idx]))
    frame(anim)
end
gif(anim)